# Run: rdrp1 Human

```
Lead     : ababaian
Issue    : 
Version  : v0.4.0-dev : diamond-dev branch
start    : 2021 01 16
complete : 2021 01 xx
files    : ~/serratus/notebook/210108_ab/
s3_files : s3://serratus-public/notebook/210108_ab/
output   : s3://serratus-public/out/210116_rMu/
```

### Intro/Objectives

> In the desert, you can remember your name 'cause there ain't no one for to give you no pain

### Mouse

- File: `mu_SraRunInfo.csv`

Search Term:
```
"Mus musculus"[orgn] AND ("type_rnaseq"[Filter] OR "metagenomic"[Filter] OR "metatranscriptomic"[Filter]) AND "platform illumina"[Properties]
```

SRA Accessed: `2020/12/30` 

Results: `1 051 719 / 1 058 559`

[See: 201230_sraRunInfo_Update.ipynb](./201230_sraRunInfo_Update.ipynb)


In [ ]:
## Fire up EC2 Instance
#sudo yum install -y docker
#sudo yum install -y git
#sudo service docker start

## Download latest serratus repo
#git clone -b diamond-dev https://github.com/ababaian/serratus.git; cd serratus/containers

## If you want to upload containers to your repository, include this.
#export DOCKERHUB_USER='serratusbio' # optional
#sudo docker login # optional
#
## Build all containers and upload them docker hub repo (if available)
#./build_containers.sh

### Initialize local workspace

In [1]:
# Serratus commit version
SERRATUS="/home/artem/serratus"
cd $SERRATUS

# Create local run directory
WORK="$SERRATUS/notebook/210111_ab"
mkdir -p $WORK; cd $WORK

# S3 notebook path
S3_WORK='s3://serratus-public/notebook/210111_ab'

# date and version
date
git rev-parse HEAD # commit version

Sat Jan 16 22:26:51 PST 2021
2ddf24166fdded0bc89a96b2541bcbc7442f98f9


In [2]:
cd $WORK

# Copy SRA Run Info
ln -s ../201230_sra/completed.sra ./
ln -s ../201230_sra/mu_SraRunInfo.csv ./

md5sum mu*
ls -alh mu*

ln: failed to create symbolic link './completed.sra': File exists
245884d3ded9056bd278f357347b53bf  mu_SraRunInfo.csv
lrwxrwxrwx 1 artem artem 31 Jan 16 22:26 mu_SraRunInfo.csv -> ../201230_sra/mu_SraRunInfo.csv


In [4]:
cd $WORK

remove_done () {
  COMPLETE=$1
  SRA=$2
  
  # Sort unique SRA in the input RunInfo file
  cut -f1 -d',' $SRA \
    | sort - \
    > inputsra.tmp
  
  # Ensure completed SRA list is sorted
  sort $COMPLETE > complete.sort.tmp
  
  # Supress SRA-accessions only in file 1 or file 1+2
  # Show SRA unique to file 2
  # parse into a regex expression to find matching lines
  comm -1 -3 complete.sort.tmp inputsra.tmp \
    | sed 's/^$\n//g' - \
    | sed 's/^/^/g' - \
    | sed 's/$/,/g' \
    > incomplete.in.$SRA
    
  grep -f incomplete.in.$SRA $SRA > undone_$SRA
  
  rm *.tmp
  # Still crashing due to high memory
  # but will give a list of "undone" accessions
}

remove_done completed.sra mu_SraRunInfo.csv # not-done: 1 056 519 / 1 058 559

Killed


In [5]:
# Chop up file for batch-runs
sed -n '1,275000p;275001q' mu_SraRunInfo.csv > mu_SraRunInfo_1.csv

head -n1 mu_SraRunInfo.csv > mu_SraRunInfo_2.csv
sed -n '275000,550000p;550001q' mu_SraRunInfo.csv >> mu_SraRunInfo_2.csv

head -n1 mu_SraRunInfo.csv > mu_SraRunInfo_3.csv
sed -n '550000,825000p;825001q' mu_SraRunInfo.csv >> mu_SraRunInfo_3.csv

head -n1 mu_SraRunInfo.csv > mu_SraRunInfo_4.csv
tail -n+825001 mu_SraRunInfo.csv >> mu_SraRunInfo_4.csv

wc -l mu_SraRunInfo_*

   275000 mu_SraRunInfo_1.csv
   275002 mu_SraRunInfo_2.csv
   275002 mu_SraRunInfo_3.csv
   233560 mu_SraRunInfo_4.csv
  1058564 total


In [ ]:
#aws s3 sync ./ $S3_WORK/

### Terraform Initialize

In [ ]:
## UPDATE serratus/terraform/main/main.tf
# main/main.tf LINE 187 = "-b s3://serratus-public/out/yymmdd_id"

In [6]:
# BATCH 1
#
# For rapid batching; copy out serratus folder
# PROTEIN / DNA MUST BE SET IN CONFIG FILE
# main/main.tf LINE 166
#   options            = "-k ${module.work_bucket.name} -a bowtie2"
#   options            = "-k ${module.work_bucket.name} -a diamond"
# Set output location
# main/main.tf LINE 187 = "-b s3://serratus-public/out/yymmdd_id"

TF=$SERRATUS/terraform/main
cd $TF
git diff main.tf
terraform init

# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve

diff --git a/terraform/main/main.tf b/terraform/main/main.tf
index de2d00d..9bd2d4f 100644
--- a/terraform/main/main.tf
+++ b/terraform/main/main.tf
@@ -12,14 +12,14 @@ variable "aws_region" {
 }
 
 variable "dl_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of downloader nodes (ASG)"
 }
 
 variable "align_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of aligner nodes (ASG)"
 }
 
@@ -38,14 +38,22 @@ variable "dockerhub_account" {
 }
 
 variable "scheduler_port" {
-  type  = number
+  type    = number
   default = 8000
 }
 
+variable "output_bucket" {
+  type = string
+}
+
+//variable "metrics_ip" {
+//  type = string
+//}
+
 // PROVIDER/AWS ##############################
 provider "aws" {
-  version     = "~> 2.49"
-  region      = var.aws_region
+  version = "~> 2.49"
+  region  = var.aws_region
 }
 
 provider "local" {
@@ -81,68 +89,73 @@ re

     #!/bin/bash
     set -eu
     ssh -Nf -L 3000:localhost:3000 -L 9090:localhost:9090 ec2-user@${module.monitoring.public_dns}
@@ -208,9 +216,9 @@ resource "local_file" "create_tunnel" {
 }
 
 resource "local_file" "upload_sra" {
-  filename = "${path.module}/uploadSRA.sh"
+  filename        = "${path.module}/uploadSRA.sh"
   file_permission = 0777
-  content = <<-EOF
+  content         = <<-EOF
     #!/bin/bash
     # =====================================
     # Serratus - uploadSRA.sh
@@ -289,7 +297,6 @@ resource "local_file" "upload_sra" {
   EOF
 }
 
-
 // OUTPUT ##############################
 output "help" {
   value = <<-EOF
Initializing modules...

Initializing the backend...

Initializing provider plugins...

The following providers do not have any version constraints in configuration,
so the latest version was installed.

To prevent automatic upgrades to new major versions that may contain breaking
changes, it is recommended to add version = "..." constraints to the
corres

module.merge.aws_iam_role_policy.ec2Describe: Creation complete after 1s [id=SerratusIamRole-serratus-merge:DescribeEC2Instances-serratus-merge]
module.scheduler.module.ecs_cluster.aws_iam_instance_profile.p: Creating...
module.monitoring.aws_iam_role_policy.cloudwatch: Creating...
module.align.module.iam_role.aws_iam_role_policy_attachment.attachment["arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"]: Creation complete after 1s [id=SerratusIamRole-serratus-align-20210117065416117700000004]
module.monitoring.aws_iam_role_policy_attachment.attachment: Creating...
module.align.module.iam_role.aws_iam_role_policy.cloudwatch: Creation complete after 1s [id=SerratusIamRole-serratus-align:CloudWatchLogsCreate-serratus-align]
module.download.module.iam_role.aws_iam_role_policy.cloudwatch: Creating...
module.scheduler.module.ecs_cluster.aws_iam_role_policy_attachment.instance_attachment: Creation complete after 1s [id=SerratusEcsInstanceRole-scheduler-20210117065416523700000005]
module.download

module.monitoring.aws_eip.monitor: Creation complete after 3s [id=eipalloc-0dd60a3de3efe112c]
module.monitoring.aws_ecs_task_definition.monitor: Creation complete after 1s [id=monitor]
module.monitoring.aws_ecs_service.monitor: Creating...
module.monitoring.aws_ecs_service.monitor: Creation complete after 1s [id=arn:aws:ecs:us-east-1:797308887321:service/serratus-monitor/serratus-monitor]
module.scheduler.aws_eip.sch: Creation complete after 3s [id=eipalloc-0bd366a5557b2d1a2]
module.align.aws_launch_configuration.worker: Creating...
local_file.hosts: Creating...
local_file.create_tunnel: Creating...
module.merge.aws_launch_configuration.worker: Creating...
local_file.hosts: Creation complete after 0s [id=a25a6bb8668bd85d8a97477f9a8d9a7024cbcdc5]
module.download.aws_launch_configuration.worker: Creating...
local_file.create_tunnel: Creation complete after 0s [id=13c42fb72fc10f2372b8d77f760ffa451af7bc26]
module.align.aws_launch_configuration.worker: Creation complete after 2s [id=serratu

In [14]:
# BATCH 2
#
# For rapid batching; copy out serratus folder
# PROTEIN / DNA MUST BE SET IN CONFIG FILE
# main/main.tf LINE 166
#   options            = "-k ${module.work_bucket.name} -a bowtie2"
#   options            = "-k ${module.work_bucket.name} -a diamond"
# Set output location
# main/main.tf LINE 187 = "-b s3://serratus-public/out/yymmdd_id"

TF=$SERRATUS/terraform/main
cd $TF
git diff main.tf
terraform init

# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve

Connection to ec2-34-194-202-70.compute-1.amazonaws.com closed by remote host.
Connection to ec2-50-19-157-63.compute-1.amazonaws.com closed by remote host.
diff --git a/terraform/main/main.tf b/terraform/main/main.tf
index de2d00d..9bd2d4f 100644
--- a/terraform/main/main.tf
+++ b/terraform/main/main.tf
@@ -12,14 +12,14 @@ variable "aws_region" {
 }
 
 variable "dl_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of downloader nodes (ASG)"
 }
 
 variable "align_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of aligner nodes (ASG)"
 }
 
@@ -38,14 +38,22 @@ variable "dockerhub_account" {
 }
 
 variable "scheduler_port" {
-  type  = number
+  type    = number
   default = 8000
 }
 
+variable "output_bucket" {
+  type = string
+}
+
+//variable "metrics_ip" {
+//  type = string
+//}
+
 // PROVIDER/AWS ##############################
 provider "aws" {


+  filename        = "${path.module}/create_tunnels.sh"
   file_permission = 0777
-  content = <<-EOF
+  content         = <<-EOF
     #!/bin/bash
     set -eu
     ssh -Nf -L 3000:localhost:3000 -L 9090:localhost:9090 ec2-user@${module.monitoring.public_dns}
@@ -208,9 +216,9 @@ resource "local_file" "create_tunnel" {
 }
 
 resource "local_file" "upload_sra" {
-  filename = "${path.module}/uploadSRA.sh"
+  filename        = "${path.module}/uploadSRA.sh"
   file_permission = 0777
-  content = <<-EOF
+  content         = <<-EOF
     #!/bin/bash
     # =====================================
     # Serratus - uploadSRA.sh
@@ -289,7 +297,6 @@ resource "local_file" "upload_sra" {
   EOF
 }
 
-
 // OUTPUT ##############################
 output "help" {
   value = <<-EOF
Initializing modules...

Initializing the backend...

Initializing provider plugins...

The following providers do not have any version constraints in configuration,
so the latest version was installed.

To prevent automatic up

module.align.module.iam_role.aws_iam_instance_profile.profile: Creation complete after 1s [id=profile-serratus-align]
module.scheduler.module.ecs_cluster.aws_iam_instance_profile.p: Creation complete after 1s [id=instance-profile-serratus-scheduler]
module.download.module.iam_role.aws_iam_role_policy_attachment.attachment["arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"]: Creating...
module.download.module.iam_role.aws_iam_instance_profile.profile: Creating...
module.merge.module.iam_role.aws_iam_role_policy_attachment.attachment["arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"]: Creation complete after 1s [id=SerratusIamRole-serratus-merge-20210117173838452600000005]
module.align.aws_iam_role_policy.AdjustAutoScaling: Creating...
module.merge.module.iam_role.aws_iam_role_policy.cloudwatch: Creation complete after 1s [id=SerratusIamRole-serratus-merge:CloudWatchLogsCreate-serratus-merge]
module.scheduler.aws_iam_role_policy.scheduler: Creating...
module.merge.module.iam_role.aws_iam_in

module.monitoring.module.ecs_cluster.aws_instance.i: Creation complete after 17s [id=i-0dbf9e2605f77b2b3]
module.monitoring.aws_eip.monitor: Creating...
module.scheduler.module.ecs_cluster.aws_instance.i: Creation complete after 17s [id=i-0ca5a69247306a0a1]
module.scheduler.aws_eip.sch: Creating...
module.monitoring.aws_ecs_task_definition.monitor: Creating...
module.monitoring.aws_ecs_task_definition.monitor: Creation complete after 1s [id=monitor]
module.monitoring.aws_ecs_service.monitor: Creating...
module.monitoring.aws_ecs_service.monitor: Creation complete after 1s [id=arn:aws:ecs:us-east-1:797308887321:service/serratus-monitor/serratus-monitor]
module.scheduler.aws_eip.sch: Creation complete after 3s [id=eipalloc-0762a330cad761bf7]
module.download.aws_launch_configuration.worker: Creating...
module.merge.aws_launch_configuration.worker: Creating...
module.align.aws_launch_configuration.worker: Creating...
module.monitoring.aws_eip.monitor: Creation complete after 3s [id=eipallo

In [28]:
# BATCH 3
#
# For rapid batching; copy out serratus folder
# PROTEIN / DNA MUST BE SET IN CONFIG FILE
# main/main.tf LINE 166
#   options            = "-k ${module.work_bucket.name} -a bowtie2"
#   options            = "-k ${module.work_bucket.name} -a diamond"
# Set output location
# main/main.tf LINE 187 = "-b s3://serratus-public/out/yymmdd_id"

TF=$SERRATUS/terraform/main
cd $TF
git diff main.tf
terraform init

# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve

client_loop: send disconnect: Broken pipe
diff --git a/terraform/main/main.tf b/terraform/main/main.tf
index de2d00d..9bd2d4f 100644
--- a/terraform/main/main.tf
+++ b/terraform/main/main.tf
@@ -12,14 +12,14 @@ variable "aws_region" {
 }
 
 variable "dl_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of downloader nodes (ASG)"
 }
 
 variable "align_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of aligner nodes (ASG)"
 }
 
@@ -38,14 +38,22 @@ variable "dockerhub_account" {
 }
 
 variable "scheduler_port" {
-  type  = number
+  type    = number
   default = 8000
 }
 
+variable "output_bucket" {
+  type = string
+}
+
+//variable "metrics_ip" {
+//  type = string
+//}
+
 // PROVIDER/AWS ##############################
 provider "aws" {
-  version     = "~> 2.49"
-  region      = var.aws_region
+  version = "~> 2.49"
+  region  = var.aws_region
 }
 


+  content         = <<-EOF
     #!/bin/bash
     set -eu
     ssh -Nf -L 3000:localhost:3000 -L 9090:localhost:9090 ec2-user@${module.monitoring.public_dns}
@@ -208,9 +216,9 @@ resource "local_file" "create_tunnel" {
 }
 
 resource "local_file" "upload_sra" {
-  filename = "${path.module}/uploadSRA.sh"
+  filename        = "${path.module}/uploadSRA.sh"
   file_permission = 0777
-  content = <<-EOF
+  content         = <<-EOF
     #!/bin/bash
     # =====================================
     # Serratus - uploadSRA.sh
@@ -289,7 +297,6 @@ resource "local_file" "upload_sra" {
   EOF
 }
 
-
 // OUTPUT ##############################
 output "help" {
   value = <<-EOF
Initializing modules...

Initializing the backend...

Initializing provider plugins...

The following providers do not have any version constraints in configuration,
so the latest version was installed.

To prevent automatic upgrades to new major versions that may contain breaking
changes, it is recommended to add version = "..

module.merge.module.iam_role.aws_iam_role_policy_attachment.attachment["arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"]: Creation complete after 1s [id=SerratusIamRole-serratus-merge-20210118035244324100000004]
module.download.module.iam_role.aws_iam_role_policy.cloudwatch: Creating...
module.align.module.iam_role.aws_iam_role_policy.cloudwatch: Creation complete after 1s [id=SerratusIamRole-serratus-align:CloudWatchLogsCreate-serratus-align]
module.merge.aws_iam_role_policy.AdjustAutoScaling: Creating...
module.align.module.iam_role.aws_iam_instance_profile.profile: Creation complete after 1s [id=profile-serratus-align]
module.merge.aws_iam_role_policy.ec2Describe: Creating...
aws_security_group.internal: Creation complete after 5s [id=sg-028db5379ed6ccbbf]
module.merge.aws_iam_role_policy.ec2Terminate: Creating...
module.align.module.iam_role.aws_iam_role_policy_attachment.attachment["arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"]: Creation complete after 1s [id=SerratusIamRole-se

module.monitoring.aws_ecs_task_definition.monitor: Creating...
module.scheduler.aws_eip.sch: Creating...
module.monitoring.aws_ecs_task_definition.monitor: Creation complete after 1s [id=monitor]
module.monitoring.aws_ecs_service.monitor: Creating...
module.monitoring.aws_ecs_service.monitor: Creation complete after 1s [id=arn:aws:ecs:us-east-1:797308887321:service/serratus-monitor/serratus-monitor]
module.monitoring.aws_eip.monitor: Creation complete after 3s [id=eipalloc-0f7af7d76fca76579]
module.scheduler.aws_eip.sch: Creation complete after 3s [id=eipalloc-08248b8674b280dcb]
local_file.create_tunnel: Creating...
local_file.hosts: Creating...
local_file.create_tunnel: Creation complete after 0s [id=8c0164afd5a966ab60c26348b7696ceedf9bc22b]
module.merge.aws_launch_configuration.worker: Creating...
local_file.hosts: Creation complete after 0s [id=f478b7a616e57d94af1af25388119a9a1e25ea02]
module.download.aws_launch_configuration.worker: Creating...
module.align.aws_launch_configuration

In [43]:
# BATCH 4
#
# For rapid batching; copy out serratus folder
# PROTEIN / DNA MUST BE SET IN CONFIG FILE
# main/main.tf LINE 166
#   options            = "-k ${module.work_bucket.name} -a bowtie2"
#   options            = "-k ${module.work_bucket.name} -a diamond"
# Set output location
# main/main.tf LINE 187 = "-b s3://serratus-public/out/yymmdd_id"

TF=$SERRATUS/terraform/main
cd $TF
git diff main.tf
terraform init

# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve

Connection to ec2-35-171-132-104.compute-1.amazonaws.com closed by remote host.
diff --git a/terraform/main/main.tf b/terraform/main/main.tf
index de2d00d..9bd2d4f 100644
--- a/terraform/main/main.tf
+++ b/terraform/main/main.tf
@@ -12,14 +12,14 @@ variable "aws_region" {
 }
 
 variable "dl_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of downloader nodes (ASG)"
 }
 
 variable "align_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of aligner nodes (ASG)"
 }
 
@@ -38,14 +38,22 @@ variable "dockerhub_account" {
 }
 
 variable "scheduler_port" {
-  type  = number
+  type    = number
   default = 8000
 }
 
+variable "output_bucket" {
+  type = string
+}
+
+//variable "metrics_ip" {
+//  type = string
+//}
+
 // PROVIDER/AWS ##############################
 provider "aws" {
-  version     = "~> 2.49"
-  region      = var.aws_region
+  version = "~> 2

   file_permission = 0777
-  content = <<-EOF
+  content         = <<-EOF
     #!/bin/bash
     set -eu
     ssh -Nf -L 3000:localhost:3000 -L 9090:localhost:9090 ec2-user@${module.monitoring.public_dns}
@@ -208,9 +216,9 @@ resource "local_file" "create_tunnel" {
 }
 
 resource "local_file" "upload_sra" {
-  filename = "${path.module}/uploadSRA.sh"
+  filename        = "${path.module}/uploadSRA.sh"
   file_permission = 0777
-  content = <<-EOF
+  content         = <<-EOF
     #!/bin/bash
     # =====================================
     # Serratus - uploadSRA.sh
@@ -289,7 +297,6 @@ resource "local_file" "upload_sra" {
   EOF
 }
 
-
 // OUTPUT ##############################
 output "help" {
   value = <<-EOF
Initializing modules...

Initializing the backend...

Initializing provider plugins...

The following providers do not have any version constraints in configuration,
so the latest version was installed.

To prevent automatic upgrades to new major versions that may contain breaking
c

module.merge.module.iam_role.aws_iam_role_policy.cloudwatch: Creation complete after 1s [id=SerratusIamRole-serratus-merge:CloudWatchLogsCreate-serratus-merge]
module.merge.module.iam_role.aws_iam_instance_profile.profile: Creation complete after 2s [id=profile-serratus-merge]
module.monitoring.module.ecs_cluster.aws_iam_instance_profile.p: Creating...
module.download.aws_iam_role_policy.AdjustAutoScaling: Creating...
module.scheduler.module.ecs_cluster.aws_iam_role_policy_attachment.instance_attachment: Creation complete after 1s [id=SerratusEcsInstanceRole-scheduler-20210118085430356800000005]
module.monitoring.module.ecs_cluster.aws_iam_role_policy_attachment.instance_attachment: Creating...
module.align.module.iam_role.aws_iam_role_policy.cloudwatch: Creation complete after 1s [id=SerratusIamRole-serratus-align:CloudWatchLogsCreate-serratus-align]
module.download.aws_iam_role_policy.ec2Terminate: Creating...
module.align.module.iam_role.aws_iam_instance_profile.profile: Creation co

module.monitoring.aws_ecs_task_definition.monitor: Creating...
module.monitoring.module.ecs_cluster.aws_instance.i: Creation complete after 16s [id=i-0322c40dfd582f116]
module.monitoring.aws_eip.monitor: Creating...
module.monitoring.aws_ecs_task_definition.monitor: Creation complete after 1s [id=monitor]
module.monitoring.aws_ecs_service.monitor: Creating...
module.monitoring.aws_ecs_service.monitor: Creation complete after 1s [id=arn:aws:ecs:us-east-1:797308887321:service/serratus-monitor/serratus-monitor]
module.scheduler.aws_eip.sch: Creation complete after 4s [id=eipalloc-0647611fdb81fa431]
module.download.aws_launch_configuration.worker: Creating...
module.align.aws_launch_configuration.worker: Creating...
module.merge.aws_launch_configuration.worker: Creating...
module.monitoring.aws_eip.monitor: Creation complete after 3s [id=eipalloc-0f2afcd603d4588c0]
local_file.create_tunnel: Creating...
local_file.hosts: Creating...
local_file.create_tunnel: Creation complete after 0s [id=9

Serratus Config backup 
```
{
  "ALIGN_ARGS": "--very-sensitive-local",
  "ALIGN_MAX_INCREASE": 40,
  "ALIGN_SCALING_CONSTANT": 0.08,
  "ALIGN_SCALING_ENABLE": true,
  "ALIGN_SCALING_MAX": 3200,
  "CLEAR_INTERVAL": 999999,
  "DL_ARGS": "",
  "DL_MAX_INCREASE": 20,
  "DL_SCALING_CONSTANT": 0.1,
  "DL_SCALING_ENABLE": true,
  "DL_SCALING_MAX": 1350,
  "GENOME": "rdrp1",
  "MERGE_ARGS": "protein",
  "MERGE_MAX_INCREASE": 25,
  "MERGE_SCALING_CONSTANT": 0.1,
  "MERGE_SCALING_ENABLE": true,
  "MERGE_SCALING_MAX": 25,
  "SCALING_INTERVAL": 120,
  "VIRTUAL_SCALING_INTERVAL": 35
}
```

In [45]:
cd $TF

# Open SSH tunnels to the monitor
./create_tunnels.sh

# If you get an error on port
# run:
# ps aux | grep ssh
# sudo kill <PID of SSH>

channel 4: open failed: connect failed: Connection refused
channel 4: open failed: connect failed: Connection refused
channel 4: open failed: connect failed: Connection refused
channel 4: open failed: connect failed: Connection refused
channel 4: open failed: connect failed: Connection refused
channel 4: open failed: connect failed: Connection refused
channel 4: open failed: connect failed: Connection refused
channel 4: open failed: connect failed: Connection refused
channel 4: open failed: connect failed: Connection refused
Tunnels created:
    localhost:3000 = grafana
    localhost:9090 = prometheus
    localhost:5432 = postgres
    localhost:8000 = scheduler


In [8]:
# BATCH 1
cd $WORK
BATCH1='mu_SraRunInfo_1.csv'
wc -l $WORK/$BATCH1

275000 /home/artem/serratus/notebook/210111_ab/mu_SraRunInfo_1.csv


In [11]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh $WORK/$BATCH1

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/210111_ab/mu_SraRunInfo_1.csv
  date: Sat Jan 16 22:58:11 PST 2021
  wc  : 275000 /home/artem/serratus/notebook/210111_ab/mu_SraRunInfo_1.csv
  md5 : 5c45dd9d599c7b0b94d655cdb020719b  /home/artem/serratus/notebook/210111_ab/mu_SraRunInfo_1.csv


--------------------------
tmp.chunk00
10001 tmp.chunk00_sraRunInfo.csv
a18c03cfa619ce7860c89938d6415d38  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":10000}
--------------------------
tmp.chunk01
10001 tmp.chunk01_sraRunInfo.csv
dc60c4deacf78e4457e423b814b59208  tmp.chunk01_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":20000}
--------------------------
tmp.chunk02
10001 tmp.chunk02_sraRunInfo.csv
997d39709cd5276959c8581903ff1ab2  tmp.chunk02_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":30000}
--------------------------
tmp.chunk03
10001 tmp.chunk03_sraRunInfo.csv
d11beb6db9ddbe3d8d639ea368ca2f14  tmp.chunk03_sraRunInfo.csv
{"inserted_rows":100

In [16]:
# BATCH 2
cd $WORK
BATCH2='mu_SraRunInfo_2.csv'
wc -l $WORK/$BATCH2

275002 /home/artem/serratus/notebook/210111_ab/mu_SraRunInfo_2.csv


In [19]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh $WORK/$BATCH2

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/210111_ab/mu_SraRunInfo_2.csv
  date: Sun Jan 17 09:44:51 PST 2021
  wc  : 275002 /home/artem/serratus/notebook/210111_ab/mu_SraRunInfo_2.csv
  md5 : b0c8ac164fbeff5b4c9406da26677007  /home/artem/serratus/notebook/210111_ab/mu_SraRunInfo_2.csv


--------------------------
tmp.chunk00
10001 tmp.chunk00_sraRunInfo.csv
fa4ff8fa3b797567557983fb764ba8dd  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":10000}
--------------------------
tmp.chunk01
10001 tmp.chunk01_sraRunInfo.csv
7f8298295c65c1e0503cd3b1aaba4f0f  tmp.chunk01_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":20000}
--------------------------
tmp.chunk02
10001 tmp.chunk02_sraRunInfo.csv
8584060dfc4d859ab6cbd68eeb83d727  tmp.chunk02_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":30000}
--------------------------
tmp.chunk03
10001 tmp.chunk03_sraRunInfo.csv
a9f4a0be76147c059e3f965bd62bbcc2  tmp.chunk03_sraRunInfo.csv
{"inserted_rows":100

In [36]:
# BATCH 3
cd $WORK
BATCH3='mu_SraRunInfo_3.csv'
wc -l $WORK/$BATCH3

275002 /home/artem/serratus/notebook/210111_ab/mu_SraRunInfo_3.csv


In [37]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh $WORK/$BATCH3

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/210111_ab/mu_SraRunInfo_3.csv
  date: Sun Jan 17 20:06:44 PST 2021
  wc  : 275002 /home/artem/serratus/notebook/210111_ab/mu_SraRunInfo_3.csv
  md5 : 725039180f4ff506d8a03f802305e507  /home/artem/serratus/notebook/210111_ab/mu_SraRunInfo_3.csv


--------------------------
tmp.chunk00
10001 tmp.chunk00_sraRunInfo.csv
16902fa89f2d50e059a1f1ad97aca86a  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":20000}
--------------------------
tmp.chunk00_sraRunInfo.csv_sraRunInfo.csv
2 tmp.chunk00_sraRunInfo.csv_sraRunInfo.csv_sraRunInfo.csv
24902b0d5bb78f860a55a84622499841  tmp.chunk00_sraRunInfo.csv_sraRunInfo.csv_sraRunInfo.csv
{"inserted_rows":1,"total_rows":20001}
--------------------------
tmp.chunk01
10001 tmp.chunk01_sraRunInfo.csv
02002d883073931d290345b3f79d461d  tmp.chunk01_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":30001}
--------------------------
tmp.chunk02
10001 tmp.chunk02_sraRunInfo.csv

In [50]:
# BATCH 4
cd $WORK
BATCH4='mu_SraRunInfo_4.csv'
wc -l $WORK/$BATCH4

233560 /home/artem/serratus/notebook/210111_ab/mu_SraRunInfo_4.csv


In [51]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh $WORK/$BATCH4

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/210111_ab/mu_SraRunInfo_4.csv
  date: Mon Jan 18 00:59:32 PST 2021
  wc  : 233560 /home/artem/serratus/notebook/210111_ab/mu_SraRunInfo_4.csv
  md5 : dbb57a8b22cab4f2ac8126e80240b47b  /home/artem/serratus/notebook/210111_ab/mu_SraRunInfo_4.csv


--------------------------
tmp.chunk00
10001 tmp.chunk00_sraRunInfo.csv
765e5657bc267f5236c663bb01b873a6  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":10000}
--------------------------
tmp.chunk01
10001 tmp.chunk01_sraRunInfo.csv
033c8202f0106d338beb10b9aa0627c1  tmp.chunk01_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":20000}
--------------------------
tmp.chunk02
10001 tmp.chunk02_sraRunInfo.csv
b16d7b9aaae87bc841b9d67c134d3ad3  tmp.chunk02_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":30000}
--------------------------
tmp.chunk03
10001 tmp.chunk03_sraRunInfo.csv
ffa0f5a5f3cc3ce55c9b6be37574470e  tmp.chunk03_sraRunInfo.csv
{"inserted_rows":100

## Run Serratus

In [54]:
# Set Cluster Parameters =============================
## get Config File (if it doesn't exist)
# curl localhost:8000/config | jq > serratus-config.json

cd $TF
# Make local changes to config file
echo "  Cluster Config File: "
cat serratus-config.json
echo ""
echo ""
# Re-upload config file
curl -sT serratus-config.json localhost:8000/config

  Cluster Config File: 
{
  "ALIGN_ARGS": "--very-sensitive-local",
  "ALIGN_MAX_INCREASE": 35,
  "ALIGN_SCALING_CONSTANT": 0.08,
  "ALIGN_SCALING_ENABLE": true,
  "ALIGN_SCALING_MAX": 1500,
  "CLEAR_INTERVAL": 999999,
  "DL_ARGS": "",
  "DL_MAX_INCREASE": 20,
  "DL_SCALING_CONSTANT": 0.1,
  "DL_SCALING_ENABLE": true,
  "DL_SCALING_MAX": 950,
  "GENOME": "rdrp1",
  "MERGE_ARGS": "protein",
  "MERGE_MAX_INCREASE": 25,
  "MERGE_SCALING_CONSTANT": 0.5,
  "MERGE_SCALING_ENABLE": true,
  "MERGE_SCALING_MAX": 40,
  "SCALING_INTERVAL": 120,
  "VIRTUAL_SCALING_INTERVAL": 35
}

{"ALIGN_ARGS":"--very-sensitive-local","ALIGN_MAX_INCREASE":35,"ALIGN_SCALING_CONSTANT":0.08,"ALIGN_SCALING_ENABLE":true,"ALIGN_SCALING_MAX":1500,"CLEAR_INTERVAL":999999,"DL_ARGS":"","DL_MAX_INCREASE":20,"DL_SCALING_CONSTANT":0.1,"DL_SCALING_ENABLE":true,"DL_SCALING_MAX":950,"GENOME":"rdrp1","MERGE_ARGS":"protein","MERGE_MAX_INCREASE":25,"MERGE_SCALING_CONSTANT":0.5,"MERGE_SCALING_ENABLE":true,"MERGE_SCALING_MAX":40,"SCA

### Error handling

In [ ]:
## Stop postgres if it's running 
# systemctl stop postgresql

## Connect to postgres
# psql -h localhost postgres postgres

#  psql -h localhost postgres postgres -c "DELETE FROM blocks WHERE state = 'done';"

### ACCESSION OPERATIONS
## Reset SPLITTING accessions to NEW
# UPDATE acc SET state = 'new' WHERE state = 'splitting';

## Reset SPLIT_ERR accessions to NEW
## (repeated failures can be missing SRA data)
# UPDATE acc SET state = 'new' WHERE state = 'split_err';

## Reset MERGE_ERR accessions to SPLIT_DONE
# UPDATE acc SET state = 'split_done' WHERE state = 'merge_err';

## Clear DONE Accessions (ONLY ON COMPLETION)
# DELETE FROM acc WHERE state = 'merge_done';

### BLOCK OPERATIONS

##  Reset FAIL blocks to NEW
# UPDATE blocks SET state = 'new' WHERE state = 'fail';

# Reset ALIGNING blocks to NEW
# UPDATE blocks SET state = 'new' WHERE state = 'aligning';

# Clear Done
# DELETE FROM blocks WHERE state = 'done';

# RESET STATE
# DELETE FROM blocks WHERE state = 'done';
# DELETE FROM blocks WHERE state = 'fail';
#
#
# DELETE FROM acc WHERE state = 'split_err';
# DELETE FROM acc WHERE state = 'merging';
# DELETE FROM acc WHERE state = 'merge_err';
# DELETE FROM acc WHERE state = 'split_done';


In [ ]:
# Regular Shutdown
cd $TF
terraform destroy

## Nuke Shutdown
#cd $TF
#
#aws ec2 describe-instances \
#  --filter Name=tag:Name,Values=serratus-align-instance \
#  > align_instances.json
#
#jq '.Reservations[].Instances[].InstanceId' -r align_instances.json \
#  | pv -l \
#  | xargs -n10 -P10 aws ec2 terminate-instances --instance-ids

# Notes: 



psummary: `https://serratus-public.s3.amazonaws.com/out/210108_r1p/psummary/<SRA>.psummary`

SRA site: `https://www.ncbi.nlm.nih.gov/sra/?term=<SRA>`
